## Download and install the dependencies

In [1]:
#Mount the google collab to the google drive to collect our data
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 18.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 83.6 

In [3]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 8.1 MB/s eta 0:00:00


In [4]:
!pip install accelerate -U


In [5]:
!nvidia-smi


Sun Nov 26 04:18:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    46W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
#Read the data file as a dataframe using pandas
import pandas as pd
file_path= '/gdrive/My Drive/Email_Text_Summarization/dialogsum.train.jsonl'
data=pd.read_json(file_path,lines=True)

In [9]:
data.head()

,fname,dialogue,summary,topic
0,train_0,"#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. ...","Mr. Smith's getting a check-up, and Doctor Haw...",get a check-up
1,train_1,"#Person1#: Hello Mrs. Parker, how have you bee...",Mrs Parker takes Ricky for his vaccines. Dr. P...,vaccines
2,train_2,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...,find keys
3,train_3,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...,have a girlfriend
4,train_4,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...,dance


In [10]:
#Install all the ncessary libraries
from transformers import pipeline, set_seed
from datasets import load_dataset, load_metric
import matplotlib.pyplot as plt
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
#Connect to GPU and set it to device, import the model and the tokenizer
device ="cuda" if torch.cuda.is_available() else "cpu"
model="google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model)
model_pegasus=AutoModelForSeq2SeqLM.from_pretrained(model).to(device)

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [12]:
#This Python function named batch takes a list of elements and a batch_size as input parameters.
#It uses a generator to slice the elements list into smaller batches of size batch_size and yields each batch successively until the entire list is iterated through.
def batch(elements, batch_size):
  for i in range(0,len(elements),batch_size):
    yield elements[i: i+batch_size]

In [13]:
#Convert the data into a pandas dataframe and find out its shape
dataset=pd.DataFrame(data)
dataset.head()
shape=dataset.shape
print(shape)

(12460, 4)


In [14]:
dataset.loc[1,"dialogue"],dataset.loc[1,"summary"]

("#Person1#: Hello Mrs. Parker, how have you been?\n#Person2#: Hello Dr. Peters. Just fine thank you. Ricky and I are here for his vaccines.\n#Person1#: Very well. Let's see, according to his vaccination record, Ricky has received his Polio, Tetanus and Hepatitis B shots. He is 14 months old, so he is due for Hepatitis A, Chickenpox and Measles shots.\n#Person2#: What about Rubella and Mumps?\n#Person1#: Well, I can only give him these for now, and after a couple of weeks I can administer the rest.\n#Person2#: OK, great. Doctor, I think I also may need a Tetanus booster. Last time I got it was maybe fifteen years ago!\n#Person1#: We will check our records and I'll have the nurse administer and the booster as well. Now, please hold Ricky's arm tight, this may sting a little.",
 'Mrs Parker takes Ricky for his vaccines. Dr. Peters checks the record and then gives Ricky a vaccine.')

In [15]:
#This Python code defines a function clean_dialogue that uses the re.sub() method from the re module to clean dialogue in a dataset.
#It searches for patterns where a speaker identifier (#Person1#, #Person2#, etc.) is followed by multiple colons and replaces them with a single colon.
#The function clean_dialogue is then applied to the 'dialogue' column of a dataset using the apply() method. Finally, it prints the cleaned 'dialogue' column for the first row of the dataset.
import re
def clean_dialogue(row):
    # Replace multiple colons after the speaker identifiers with a single colon
    dialogue_cleaned = re.sub(r'#(Person\d+)#:+', r'\1:', row)
    return dialogue_cleaned

dataset['dialogue'] = dataset['dialogue'].apply(clean_dialogue)

# Display the cleaned 'dialogue' column for the first row
print(dataset['dialogue'])

0        Person1: Hi, Mr. Smith. I'm Doctor Hawkins. Wh...
1        Person1: Hello Mrs. Parker, how have you been?...
2        Person1: Excuse me, did you see a set of keys?...
3        Person1: Why didn't you tell me you had a girl...
4        Person1: Watsup, ladies! Y'll looking'fine ton...
                               ...                        
12455    Person1: Excuse me. You are Mr. Green from Man...
12456    Person1: Mister Ewing said we should show up a...
12457    Person1: How can I help you today?\nPerson2: I...
12458    Person1: You look a bit unhappy today. What's ...
12459    Person1: Mom, I'm flying to visit uncle Lee's ...
Name: dialogue, Length: 12460, dtype: object


In [16]:
dataset.head()

,fname,dialogue,summary,topic
0,train_0,"Person1: Hi, Mr. Smith. I'm Doctor Hawkins. Wh...","Mr. Smith's getting a check-up, and Doctor Haw...",get a check-up
1,train_1,"Person1: Hello Mrs. Parker, how have you been?...",Mrs Parker takes Ricky for his vaccines. Dr. P...,vaccines
2,train_2,"Person1: Excuse me, did you see a set of keys?...",#Person1#'s looking for a set of keys and asks...,find keys
3,train_3,Person1: Why didn't you tell me you had a girl...,#Person1#'s angry because #Person2# didn't tel...,have a girlfriend
4,train_4,"Person1: Watsup, ladies! Y'll looking'fine ton...",Malik invites Nikki to dance. Nikki agrees if ...,dance


In [17]:
#The code utilizes a Hugging Face pipeline for text summarization, summarizing the content present in the 'dialogue' column of a dataset at index 1 using a pre-trained summarization model, and stores the generated summary in the variable pipe_out.
pipe=pipeline('summarization',model=model)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
pipe_out=pipe(dataset["dialogue"][1])
print(pipe_out)

[{'summary_text': 'Ricky Parker is 14 months old .<n>According to his vaccination record, Ricky has received his Polio, Tetanus and Hepatitis B shots .<n>He is due for Hepatitis A, Chickenpox and Measles shots .'}]


In [19]:
cleaned=pipe_out[0]['summary_text'].replace(" .", ".\n")
summary_text_cleaned = cleaned.replace("<n>", "\n")
print(summary_text_cleaned)

Ricky Parker is 14 months old.

According to his vaccination record, Ricky has received his Polio, Tetanus and Hepatitis B shots.

He is due for Hepatitis A, Chickenpox and Measles shots.



In [20]:
from datasets import DatasetDict, Dataset

# Assuming your DataFrame is named 'your_dataframe'
# Replace this with your actual DataFrame name

  # Your DataFrame here
dataset = dataset.drop(columns=['topic'])
dataset = dataset.rename(columns={'fname': 'id'})

# Split your_dataframe into train, test, and validation subsets
train_data = dataset[:10000]  # Example: First 10000 rows for training
test_data = dataset[10000:11000]  # Example: Next 819 rows for testing
validation_data = dataset[11000:12000]  # Example: Remaining rows for validation

# Convert each subset into a Dataset
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)
validation_dataset = Dataset.from_pandas(validation_data)

# Create a DatasetDict with train, test, and validation keys
formatted_dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
    'validation': validation_dataset
})

formatted_dataset_dict

# Access individual subsets:
# train_subset = formatted_dataset_dict['train']
# test_subset = formatted_dataset_dict['test']
# validation_subset = formatted_dataset_dict['validation']


DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 1000
    })
})

In [21]:
#This function 'convert_examples_to_features' encodes batches of dialogues and summaries using a tokenizer, generating input encodings for dialogues with a maximum length of 1024 and target encodings for summaries with a maximum length of 128.
#It returns dictionaries containing input IDs, attention masks, and labels corresponding to the encoded dialogues and summaries, facilitating model training or evaluation for sequence-to-sequence tasks like summarization.
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

# processed_data = []

# for index, row in dataset.iterrows():
#     processed_data.append(convert_examples_to_features(row))

dataset_samsum_pt = formatted_dataset_dict.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [22]:
dataset_samsum_pt

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
})

## Training
Performed Data Collation and Training the dialogsum dataset on various training arguements

In [23]:
#We Utilize the DataCollatorForSeq2Seq class from the Transformers library, initializing a data collator specifically designed for sequence-to-sequence tasks.
#It uses the provided tokenizer and model_bart to prepare the data for training or evaluation of sequence-to-sequence models like BART (Bidirectional and Auto-Regressive Transformers).
#This collator is used to batch and preprocess data for training or evaluation in sequence-to-sequence tasks such as text summarization or translation.
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [26]:
%cd /gdrive/My Drive/Email_Text_Summarization/Model

/gdrive/My Drive/Email_Text_Summarization/Model


In [27]:
#We set up a Trainer object for fine-tuning a BART model (model_bart) using specified training arguments (trainer_args), tokenizer, and data collator (seq2seq_data_collator).
#It utilizes the provided training and validation datasets (dataset_samsum_pt["train"] and dataset_samsum_pt["validation"]) for training and evaluation, respectively, then initiates the training process using the train() method.
#This Trainer configuration orchestrates fine-tuning a BART model on a specific dataset with defined training settings and conducts the training process in accordance with the specified parameters and datasets.
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=4, per_device_eval_batch_size=4,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"],
                  eval_dataset=dataset_samsum_pt["validation"])



In [28]:
trainer.train()


You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


TrainOutput(global_step=156, training_loss=2.5355449395302014, metrics={'train_runtime': 469.8206, 'train_samples_per_second': 21.285, 'train_steps_per_second': 0.332, 'total_flos': 7729633343176704.0, 'train_loss': 2.5355449395302014, 'epoch': 1.0})

In [29]:
model_pegasus.save_pretrained("pegasus-samsum-model")
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

## Inference
Testing the results saved on 3 test and 1 custom dialog between 2 people.

In [30]:

sample_text1 = formatted_dataset_dict["test"][0]["dialogue"]

reference1 = formatted_dataset_dict["test"][0]["summary"]

sample_text2 = formatted_dataset_dict["test"][1]["dialogue"]

reference2 = formatted_dataset_dict["test"][1]["summary"]

sample_text3 = formatted_dataset_dict["test"][2]["dialogue"]

reference3 = formatted_dataset_dict["test"][2]["summary"]


In [31]:
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}

pipe = pipeline("summarization", model="pegasus-samsum-model",tokenizer=tokenizer)

In [32]:

custom_dialogue="""
#Person1#: Excuse me, did you see a set of keys?
#Person2#: What kind of keys?
#Person1#: Five keys and a small foot ornament.
#Person2#: What a shame! I didn't see them.
#Person1#: Well, can you help me look for it? That's my first time here.
#Person2#: Sure. It's my pleasure. I'd like to help you look for the missing keys.
#Person1#: It's very kind of you.#Person2#: It's not a big deal.Hey, I found them.
#Person1#: Oh, thank God! I don't know how to thank you, guys.
#Person2#: You're welcome.
"""
print(pipe(custom_dialogue, **gen_kwargs))

[{'summary_text': "#Person1# asks #Person2# to help him look for five keys and a foot ornament. #Person2# says it's not a big deal."}]


In [33]:

print("Dialogue:")
print(sample_text1)


print("\nReference Summary:")
print(reference1)


print("\nModel Summary:")
print(pipe(sample_text1, **gen_kwargs)[0]["summary_text"])

Dialogue:
Person1: Are things still going badly with your house guest? 
Person2: It's getting worse. Now he's eating me out of house and home. I've tried talking to him but it all goes in one ear and out the other. He makes himself at home, which is fine. But what really gets me is that yesterday he walked into the living room in the raw and I had company over! That was the last straw. 
Person1: leo, I really think you're beating around the bush with this guy. I know he used to be your best friend in college, but I really think it's time to lay down the law. 
Person2: You're right. Everything is probably going to come to a head tonight. I'll keep you informed. 

Reference Summary:
Leo tells #Person1# things are getting worse with his house guest, who used to be his best friend in college. #Person1# suggests that it's time to lay down the law.

Model Summary:
#Person2# tells #Person1# that #Person2#'s house guest is eating her out of house and home. #Person1# thinks #Person2# should lay

In [34]:

print("Dialogue:")
print(sample_text2)


print("\nReference Summary:")
print(reference2)


print("\nModel Summary:")
print(pipe(sample_text2, **gen_kwargs)[0]["summary_text"])

Your max_length is set to 128, but your input_length is only 74. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=37)


Dialogue:
Person1: They'll do the operation for you this Friday. 
Person2: But Saturday is my birthday. 
Person1: I think it's important to do now. We can have a birthday party for you when you come out of the hospital. 
Person2: But it won't be on my birthday. 
Person1: But your health is more important. Believe me! 

Reference Summary:
#Person1#'s trying to persuade #Person2# that having the operation done is more important than #Person2#'s birthday party.

Model Summary:
#Person1# and #Person2# talk about #Person1#'s birthday and #Person2#'s health. #Person1# says #Person2#'s health is more important than #Person1#'s birthday.


In [35]:

print("Dialogue:")
print(sample_text3)


print("\nReference Summary:")
print(reference3)


print("\nModel Summary:")
print(pipe(sample_text3, **gen_kwargs)[0]["summary_text"])

Dialogue:
Person1: It's difficult to find a suitable job this year. It's a lot of pressure.
Person2: Now college graduates are being encouraged to start their own business. It sounds really good for us. We can get some valuable social experience, which will be helpful in our future jobs. We may also learn about ourselves and society.
Person1: Yes, but it's not as easy as you imagine. There are a lot of problems. The main difficulty is money and our education doesn't provide enough knowledge on how to do it.
Person2: Maybe you are right. But our government allows college graduates to borrow money from banks and maybe some can get money from their parents.
Person1: But you have to pay back the money sooner or later. So most college graduates prefer to apply for a job rather than start their own business.
Person2: But I have made up my mind to start my own business. How about you?
Person1: Well, it always takes time to consider before making a decision.

Reference Summary:
#Person2# has d


**Evaluation Metrics using Rouge Score**

**ROUGE**(Recall-Oriented Understudy for Gisting Evaluation) is a set of metrics used to evaluate automatic summarization tasks by comparing the generated summaries against reference summaries or ground truth summaries. The most commonly used ROUGE metrics are ROUGE-N, ROUGE-L, and ROUGE-W.

**ROUGE-N:** Measures the n-gram overlap between the generated summary and the reference summary. 'N' represents the length of the n-gram, such as unigrams (ROUGE-1), bigrams (ROUGE-2), etc. ROUGE-1 and ROUGE-2 are particularly popular.

**ROUGE-L:** Calculates the longest common subsequence (LCS) between the generated summary and the reference summary. It measures the longest sequence of words that are similar between the two summaries.

**ROUGE-W:** Considers weighted LCS-based statistics to compute the overlap between the generated summary and the reference summary. It penalizes longer gaps between words in the LCS.

In [37]:
formatted_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 1000
    })
})

In [39]:
#Batching Data: The function starts by dividing the input dataset into batches of articles (input text) and corresponding target summaries using a helper function batch.
#Iterating Through Batches: It iterates through the article and target summary batches, tokenizes the text using the provided tokenizer, and generates summaries using the specified model.The generated summaries are then decoded into readable text from token IDs using the tokenizer.
#Adding Predictions and References to the Metric:The decoded summaries are added to the metric object along with the reference summaries (from the dataset) for the purpose of evaluating the ROUGE scores.
#Computing ROUGE Scores:After processing all batches, the function computes the ROUGE scores using the compute() method of the metric object.
#Returning the ROUGE Scores: Finally, the function returns the computed ROUGE scores.

def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(batch(dataset[column_text], batch_size))
    target_batches = list(batch(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score


rouge_metric = load_metric('rouge')

score = calculate_metric_on_test_ds(formatted_dataset_dict['test'], rouge_metric, model_pegasus, tokenizer, column_text = 'dialogue', column_summary='summary', batch_size=8)

100%|██████████| 125/125 [07:08<00:00,  3.43s/it]


In [40]:

rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )

,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.017175,0.000263,0.017042,0.017027
